In [1]:
import pandas as pd
from datasetsforecast.m3 import M3
from neuralforecast.models import NHITS
from neuralforecast import NeuralForecast
from neuralforecast.losses.pytorch import MQLoss, DistributionLoss, PMM, GMM, NBMM

In [2]:
df, _, _ = M3.load(directory='./', group='Monthly')
#df = pd.read_excel('./m3/datasets/M3C.xls')
df['ds'] = pd.to_datetime(df['ds'])


100%|██████████| 1.76M/1.76M [00:00<00:00, 2.53MiB/s]
INFO:datasetsforecast.utils:Successfully downloaded M3C.xls, 1757696, bytes.
d:\anaconda\envs\investigacao\lib\site-packages\datasetsforecast\m3.py:108: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  freq = pd.tseries.frequencies.to_offset(class_group.freq)


In [3]:
df

,unique_id,ds,y
0,M1,1990-01-31,2640.0
1,M1,1990-02-28,2640.0
2,M1,1990-03-31,2160.0
3,M1,1990-04-30,4200.0
4,M1,1990-05-31,3360.0
...,...,...,...
167557,M999,1993-10-31,5225.9
167558,M999,1993-11-30,5236.3
167559,M999,1993-12-31,5186.6
167560,M999,1994-01-31,5143.4


In [4]:
# number of horizons
test_size = 2
val_size = 2
horizon = 12

# parametros alterados para correr mais rapido
models = [NHITS(h=horizon,
                input_size=24,
                #loss=DistributionLoss(distribution='StudentT', level=[80, 90], return_params=True),
                loss=DistributionLoss(distribution='Normal', level=[80, 90], return_params=True),
                # loss=DistributionLoss(distribution='Poisson', level=[80, 90], return_params=True),
                # loss=DistributionLoss(distribution='Tweedie', level=[80, 90], rho=1.5),
                # loss=DistributionLoss(distribution='NegativeBinomial', level=[80, 90], return_params=True),
                # loss=NBMM(n_components=2, level=[80,90]),
                # loss=GMM(n_components=2, level=[80,90]),
                # loss=PMM(n_components=1, level=[80,90]),
                max_steps=10,
                early_stop_patience_steps=20,
                # inference_windows_batch_size=1,
                val_check_steps=100,
                accelerator='cpu')]


d:\anaconda\envs\investigacao\lib\site-packages\pytorch_lightning\utilities\parsing.py:199: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
INFO:lightning_fabric.utilities.seed:Seed set to 1


In [5]:
nf = NeuralForecast(models=models, freq='M')


In [6]:
cv_nf = nf.cross_validation(df=df,
                            val_size=horizon * val_size,
                            test_size=horizon * test_size,
                            n_windows=None)

d:\anaconda\envs\investigacao\lib\site-packages\neuralforecast\common\_base_model.py:204: UserWarning: val_check_steps is greater than max_steps,                     setting val_check_steps to max_steps
  warnings.warn(
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name         | Type             | Params
--------------------------------------------------
0 | loss         | DistributionLoss | 5     
1 | padder_train | ConstantPad1d    | 0     
2 | scaler       | TemporalNorm     | 0     
3 | blocks       | ModuleList       | 2.4 M 
--------------------------------------------------
2.4 M     Trainable params
5         Non-trainable params
2.4 M     Total params
9.794

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

d:\anaconda\envs\investigacao\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
d:\anaconda\envs\investigacao\lib\site-packages\pytorch_lightning\utilities\data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 32. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
d:\anaconda\envs\investigacao\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
d:\anaconda\envs\investigacao\lib\site-packages\pytorch_lightning\loops\fit_loop.py:298: The number of training batches (45) is 

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

d:\anaconda\envs\investigacao\lib\site-packages\pytorch_lightning\utilities\data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 20. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=10` reached.
INFO:pytorch_lightning.utilities.rank_zero:Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
d:\anaconda\envs\investigacao\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:441: The 'pr

Predicting: |          | 0/? [00:00<?, ?it/s]

d:\anaconda\envs\investigacao\lib\site-packages\neuralforecast\core.py:184: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


In [7]:
cv_nf

,ds,cutoff,NHITS,NHITS-median,NHITS-lo-90,NHITS-lo-80,NHITS-hi-80,NHITS-hi-90,NHITS-loc,NHITS-scale,y
unique_id,,,,,,,,,,,
M1,1993-09-30,1993-08-31,3787.715088,3865.569336,-1984.011475,-678.531738,8122.185059,9215.059570,3882.369141,3445.741943,4800.0
M1,1993-10-31,1993-08-31,4017.407959,3918.295410,-3387.432617,-1854.405884,9726.979492,11844.321289,3723.553955,4585.217285,3000.0
M1,1993-11-30,1993-08-31,4451.931152,4487.392578,-522.838135,594.881348,8292.529297,9479.101562,4258.590820,3178.063721,3120.0
M1,1993-12-31,1993-08-31,3908.662354,3882.971680,-641.837097,230.536560,7573.639160,8620.894531,3970.807129,2775.084717,5880.0
M1,1994-01-31,1993-08-31,4284.205566,4264.413086,557.892883,1384.511108,7079.345703,8024.158691,4219.219727,2297.440430,2640.0
...,...,...,...,...,...,...,...,...,...,...,...
M999,1993-10-31,1993-02-28,5987.970215,6176.580078,-5713.567871,-2999.612061,15165.149414,17487.521484,5838.636719,6946.899902,5225.9
M999,1993-11-30,1993-02-28,7174.464844,6957.297852,-6376.321777,-3812.836182,17795.529297,20714.767578,6812.870605,8175.498047,5236.3
M999,1993-12-31,1993-02-28,6246.112305,6176.268555,-2852.093994,-939.890625,13468.412109,15611.897461,6574.448242,5776.059082,5186.6
